In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

In [2]:
results_df=pd.read_csv(r'raw02.csv')
results_df.head()

,addr_pct_cd,cmplnt_fr_dt,cmplnt_fr_tm,crm_atpt_cptd_cd,juris_desc,ky_cd,law_cat_cd,loc_of_occur_desc,ofns_desc,rpt_dt,...,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,boro_nm,patrol_boro,pd_cd,pd_desc,prem_typ_desc,cmplnt_to_dt,cmplnt_to_tm
0,69,2021-12-17T00:00:00.000,22:13:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-17T00:00:00.000,...,2.0,25.0,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,113,2021-12-17T00:00:00.000,06:21:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-17T00:00:00.000,...,3.0,46.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,113,2021-12-13T00:00:00.000,20:05:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-13T00:00:00.000,...,3.0,46.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,2021-12-07T00:00:00.000,22:49:00,COMPLETED,N.Y. POLICE DEPT,118,FELONY,NaN,DANGEROUS WEAPONS,2021-12-07T00:00:00.000,...,5.0,40.0,33.0,BRONX,PATROL BORO BRONX,792.0,WEAPONS POSSESSION 1 & 2,STREET,NaN,NaN
4,113,2021-12-06T00:00:00.000,17:25:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,2021-12-06T00:00:00.000,...,3.0,6.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
results_df=results_df[['cmplnt_fr_dt','cmplnt_fr_tm','law_cat_cd','ofns_desc','latitude','longitude','boro_nm']]
results_df.head()

,cmplnt_fr_dt,cmplnt_fr_tm,law_cat_cd,ofns_desc,latitude,longitude,boro_nm
0,2021-12-17T00:00:00.000,22:13:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.645647,-73.902876,NaN
1,2021-12-17T00:00:00.000,06:21:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.682499,-73.787269,NaN
2,2021-12-13T00:00:00.000,20:05:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.689218,-73.791449,NaN
3,2021-12-07T00:00:00.000,22:49:00,FELONY,DANGEROUS WEAPONS,40.882718,-73.881247,BRONX
4,2021-12-06T00:00:00.000,17:25:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.696483,-73.776680,NaN


In [4]:
def convertClassToNum(p_df):
    p_df['datetime']=pd.to_datetime(p_df['cmplnt_fr_dt'] + ' ' + p_df['cmplnt_fr_tm'],errors='coerce')
    p_df=p_df.drop(columns=['cmplnt_fr_dt','cmplnt_fr_tm'])
    p_df=p_df.sort_values(by='datetime',ascending=True) 

    p_df['year']=pd.DatetimeIndex(p_df['datetime']).year
    p_df['month'] = pd.DatetimeIndex(p_df['datetime']).month
    p_df['weekday'] = p_df['datetime'].dt.dayofweek
    p_df['Hour'] = p_df.datetime.dt.hour
    p_df['Minute'] = p_df.datetime.dt.minute
    p_df=p_df.drop(columns=['datetime'])

    list_law_cat_cd=list(p_df['law_cat_cd'].unique())
    global dict_law_cat_cd
    dict_law_cat_cd = {}
    for i in list_law_cat_cd:
        dict_law_cat_cd[i] = list_law_cat_cd.index(i)
    p_df['law_cat_cd']=p_df['law_cat_cd'].map(dict_law_cat_cd)
#     print(dict_law_cat_cd)

    list_ofns_desc=list(p_df['ofns_desc'].unique())
    global dict_ofns_desc
    dict_ofns_desc = {}
    for i in list_ofns_desc:
        dict_ofns_desc[i] = list_ofns_desc.index(i)
    p_df['ofns_desc']=p_df['ofns_desc'].map(dict_ofns_desc)
#     print(dict_ofns_desc)

    p_df=p_df[p_df['year'].notna()]
    p_df=p_df.fillna("OTHERS")
    global dict_boro_nm
    dict_boro_nm = {'OTHERS':0,'BROOKLYN':1,'QUEENS':2,'BRONX':3,'MANHATTAN':4,'STATEN ISLAND':5}
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm)
#     print(dict_boro_nm)

    return p_df


In [5]:
df_base = convertClassToNum(results_df) 

print(df_base.describe())
df_base.head()

          law_cat_cd      ofns_desc       latitude      longitude  \
count  449498.000000  449498.000000  449498.000000  449498.000000   
mean        0.836549       7.503924      40.736834     -73.923447   
std         0.686656       7.678109       0.082496       0.076968   
min         0.000000       0.000000      40.499401     -74.253767   
25%         0.000000       2.000000      40.675213     -73.971681   
50%         1.000000       4.000000      40.735019     -73.926464   
75%         1.000000      10.000000      40.811485     -73.879894   
max         2.000000      62.000000      40.912723     -73.700568   

             boro_nm           year          month        weekday  \
count  449498.000000  449498.000000  449498.000000  449498.000000   
mean        2.541544    2020.944950       6.823114       2.974759   
std         1.253370       0.880155       3.406422       1.975478   
min         0.000000    1920.000000       1.000000       0.000000   
25%         1.000000    2021.0000

,law_cat_cd,ofns_desc,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute
83454,0,0,40.723009,-73.978088,4,1920.0,4.0,4.0,0.0,0.0
446163,1,1,40.748923,-73.892230,2,1921.0,1.0,6.0,8.0,50.0
446108,1,2,40.652309,-73.956290,1,1921.0,1.0,3.0,22.0,0.0
389152,1,3,40.840984,-73.925974,3,1921.0,2.0,0.0,23.0,47.0
375382,0,4,40.758800,-73.890222,2,1921.0,3.0,2.0,4.0,4.0


In [6]:
df_columns = list()
for col in df_base.columns:
    df_columns.append(col)

print(df_columns)

['law_cat_cd', 'ofns_desc', 'latitude', 'longitude', 'boro_nm', 'year', 'month', 'weekday', 'Hour', 'Minute']


In [7]:
# inp = []
x = df_base.drop(['law_cat_cd', 'ofns_desc'],axis=1).values
y = df_base.get(['law_cat_cd', 'ofns_desc']).values

In [8]:
x

array([[ 40.7230091 , -73.97808829,   4.        , ...,   4.        ,
          0.        ,   0.        ],
       [ 40.74892339, -73.89223005,   2.        , ...,   6.        ,
          8.        ,  50.        ],
       [ 40.65230855, -73.95629004,   1.        , ...,   3.        ,
         22.        ,   0.        ],
       ...,
       [ 40.75484922, -73.98411999,   4.        , ...,   4.        ,
         23.        ,  40.        ],
       [ 40.62944259, -73.96707691,   1.        , ...,   4.        ,
         23.        ,  48.        ],
       [ 40.75484922, -73.98411999,   4.        , ...,   4.        ,
         23.        ,  50.        ]])

In [9]:
y

array([[ 0,  0],
       [ 1,  1],
       [ 1,  2],
       ...,
       [ 0,  4],
       [ 0, 16],
       [ 0,  4]], dtype=int64)

In [10]:
# Split the data into x_train and y_train data sets
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, \
                                                    random_state=0) # shuffle=False)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(359598, 8)
(359598, 2)
(89900, 8)
(89900, 2)


In [11]:
# Configure the XGBoost model
param = {'booster': 'gbtree', 
         'tree_method': 'gpu_hist',
         'n_estimators':100,
         'predictor': 'gpu_predictor',
         'max_depth': 50, 
         'eta': 0.1, 
         'objective': '{multi:softmax}', 
         'eval_metric': 'mlogloss',       # merror
         'num_round': 10,
         'feature_selector ': 'cyclic'
        }

In [12]:
def getPredictionResults(param, x_train, y_train, x_test, y_test, index):
    
    y_train = y_train.transpose()[index]
    y_test = y_test.transpose()[index]
    
    global xgb_clf 
    xgb_clf = XGBClassifier(param)
    xgb_clf.fit(x_train, y_train)
    score = xgb_clf.score(x_test, y_test)
    print("---------------------------------Index = {}-----------------------------------".format(index))
    print(score)

    xgbCpred = xgb_clf.predict(x_test)
    print(xgbCpred)

    print(y_test)

    results_log = classification_report(y_test, xgbCpred)
    print(results_log)
    
    return xgbCpred


In [13]:
law_cat_cd_pred = getPredictionResults(param, x_train, y_train, x_test, y_test, 0)

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:48:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------Index = 0-----------------------------------
0.512146829810901
[1 0 1 ... 1 1 1]
[1 2 0 ... 1 1 0]
              precision    recall  f1-score   support

           0       0.49      0.14      0.22     29890
           1       0.51      0.92      0.66     45116
           2       0.46      0.01      0.01     14894

    accuracy                           0.51     89900
   macro avg       0.49      0.36      0.30     89900
weighted avg       0.50      0.51      0.41     89900



In [14]:
ofns_desc_pred = getPredictionResults(param, x_train, y_train, x_test, y_test, 1)

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:49:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------Index = 1-----------------------------------
0.27545050055617354
[7 1 7 ... 4 1 1]
[ 3  7 25 ...  1  1  4]
              precision    recall  f1-score   support

           0       0.27      0.06      0.09       840
           1       0.33      0.64      0.43     17485
           2       0.23      0.12      0.16      9018
           3       0.19      0.17      0.18      9581
           4       0.30      0.17      0.22      8157
           5       0.21      0.01      0.02      3442
           6       0.32      0.02      0.05      1686
           7       0.22      0.48      0.31     14792
           8       0.74      0.94      0.82      1322
        

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [15]:
x_test

array([[ 40.69387551, -73.99920089,   1.        , ...,   5.        ,
         19.        ,  10.        ],
       [ 40.70815863, -73.9424823 ,   1.        , ...,   6.        ,
          0.        ,   1.        ],
       [ 40.59646486, -73.7819547 ,   2.        , ...,   3.        ,
         22.        ,  50.        ],
       ...,
       [ 40.72393888, -74.00245832,   4.        , ...,   3.        ,
         15.        ,  31.        ],
       [ 40.80257957, -73.96795208,   4.        , ...,   4.        ,
         11.        ,  33.        ],
       [ 40.60673687, -74.00177694,   1.        , ...,   6.        ,
         16.        ,  31.        ]])

In [16]:
data_pred = pd.DataFrame({'latitude':x_test[:,0] , 'longitude':x_test[:,1] , 'boro_nm':x_test[:,2] , 'year':x_test[:,3] , \
                          'month':x_test[:,4] , 'weekday':x_test[:,5] , 'Hour':x_test[:,6] , 'Minute':x_test[:,7] , \
                          'law_cat_cd_pred':law_cat_cd_pred , 'ofns_desc_pred': ofns_desc_pred})

In [17]:
print(data_pred.describe())
data_pred.head(20)

           latitude     longitude       boro_nm          year         month  \
count  89900.000000  89900.000000  89900.000000  89900.000000  89900.000000   
mean      40.736478    -73.923759      2.540200   2020.948877      6.821502   
std        0.082421      0.077007      1.256039      0.670725      3.410088   
min       40.500629    -74.252851      0.000000   1981.000000      1.000000   
25%       40.674993    -73.972178      1.000000   2021.000000      4.000000   
50%       40.734226    -73.926630      2.000000   2021.000000      7.000000   
75%       40.810877    -73.880320      4.000000   2021.000000     10.000000   
max       40.912723    -73.700720      5.000000   2021.000000     12.000000   

            weekday          Hour        Minute  law_cat_cd_pred  \
count  89900.000000  89900.000000  89900.000000     89900.000000   
mean       2.969121     13.126941     20.231402         0.905384   
std        1.977043      6.469003     18.644208         0.299820   
min        0.000

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
0,40.693876,-73.999201,1.0,2021.0,5.0,5.0,19.0,10.0,1,7
1,40.708159,-73.942482,1.0,2020.0,11.0,6.0,0.0,1.0,0,1
2,40.596465,-73.781955,2.0,2021.0,4.0,3.0,22.0,50.0,1,7
3,40.595426,-74.164522,5.0,2021.0,3.0,4.0,11.0,20.0,1,7
4,40.754699,-73.993501,4.0,2021.0,1.0,0.0,9.0,0.0,1,4
5,40.730066,-73.980615,4.0,2021.0,6.0,0.0,12.0,0.0,1,1
6,40.816075,-73.917645,3.0,2021.0,9.0,3.0,15.0,10.0,0,2
7,40.655166,-73.909951,1.0,2021.0,5.0,3.0,21.0,10.0,1,3
8,40.835477,-73.903217,3.0,2021.0,2.0,3.0,7.0,15.0,1,7
9,40.714382,-73.776656,2.0,2021.0,6.0,3.0,12.0,20.0,1,7


In [18]:
data_pred["boro_nm"].unique()

array([1., 2., 5., 4., 3., 0.])

In [19]:
def convertNumToClass(p_df):
    dict_law_cat_cd1 = dict(zip(dict_law_cat_cd.values(), dict_law_cat_cd.keys()))
    p_df['law_cat_cd_pred']=p_df['law_cat_cd_pred'].astype(int)
    p_df['law_cat_cd_pred']=p_df['law_cat_cd_pred'].map(dict_law_cat_cd1)
    print(dict_law_cat_cd1)

    dict_ofns_desc1 = dict(zip(dict_ofns_desc.values(), dict_ofns_desc.keys()))
    p_df['ofns_desc_pred']=p_df['ofns_desc_pred'].astype(int)
    p_df['ofns_desc_pred']=p_df['ofns_desc_pred'].map(dict_ofns_desc1)
    print(dict_ofns_desc1)

    dict_boro_nm1 = dict(zip(dict_boro_nm.values(), dict_boro_nm.keys()))
    p_df['boro_nm']=p_df['boro_nm'].astype(int)
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm1)
    print(dict_boro_nm1)

    return p_df

In [20]:
data_pred = convertNumToClass(data_pred)
print(data_pred.head())

{0: 'FELONY', 1: 'MISDEMEANOR', 2: 'VIOLATION'}
{0: 'THEFT-FRAUD', 1: 'PETIT LARCENY', 2: 'CRIMINAL MISCHIEF & RELATED OF', 3: 'ASSAULT 3 & RELATED OFFENSES', 4: 'GRAND LARCENY', 5: 'OFF. AGNST PUB ORD SENSBLTY &', 6: 'VEHICLE AND TRAFFIC LAWS', 7: 'HARRASSMENT 2', 8: 'SEX CRIMES', 9: 'RAPE', 10: 'MISCELLANEOUS PENAL LAW', 11: 'MURDER & NON-NEGL. MANSLAUGHTER', 12: 'FRAUDS', 13: 'FORGERY', 14: 'BURGLARY', 15: 'UNAUTHORIZED USE OF A VEHICLE', 16: 'FELONY ASSAULT', 17: 'OTHER STATE LAWS (NON PENAL LA', 18: 'KIDNAPPING & RELATED OFFENSES', 19: 'ROBBERY', 20: 'DANGEROUS DRUGS', 21: 'GRAND LARCENY OF MOTOR VEHICLE', 22: 'OTHER OFFENSES RELATED TO THEF', 23: 'NYS LAWS-UNCLASSIFIED FELONY', 24: 'PROSTITUTION & RELATED OFFENSES', 25: 'DANGEROUS WEAPONS', 26: 'OFFENSES AGAINST THE PERSON', 27: 'POSSESSION OF STOLEN PROPERTY', 28: 'FELONY SEX CRIMES', 29: 'ARSON', 30: 'OFFENSES INVOLVING FRAUD', 31: 'OFFENSES AGAINST PUBLIC ADMINI', 32: 'CRIMINAL TRESPASS', 33: 'INTOXICATED & IMPAIRED DRIVING', 

In [21]:
data_pred.head(20)

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
0,40.693876,-73.999201,BROOKLYN,2021.0,5.0,5.0,19.0,10.0,MISDEMEANOR,HARRASSMENT 2
1,40.708159,-73.942482,BROOKLYN,2020.0,11.0,6.0,0.0,1.0,FELONY,PETIT LARCENY
2,40.596465,-73.781955,QUEENS,2021.0,4.0,3.0,22.0,50.0,MISDEMEANOR,HARRASSMENT 2
3,40.595426,-74.164522,STATEN ISLAND,2021.0,3.0,4.0,11.0,20.0,MISDEMEANOR,HARRASSMENT 2
4,40.754699,-73.993501,MANHATTAN,2021.0,1.0,0.0,9.0,0.0,MISDEMEANOR,GRAND LARCENY
5,40.730066,-73.980615,MANHATTAN,2021.0,6.0,0.0,12.0,0.0,MISDEMEANOR,PETIT LARCENY
6,40.816075,-73.917645,BRONX,2021.0,9.0,3.0,15.0,10.0,FELONY,CRIMINAL MISCHIEF & RELATED OF
7,40.655166,-73.909951,BROOKLYN,2021.0,5.0,3.0,21.0,10.0,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES
8,40.835477,-73.903217,BRONX,2021.0,2.0,3.0,7.0,15.0,MISDEMEANOR,HARRASSMENT 2
9,40.714382,-73.776656,QUEENS,2021.0,6.0,3.0,12.0,20.0,MISDEMEANOR,HARRASSMENT 2


In [22]:
def filter_data(data_pred, user_input = None):
    if user_input == None:
        return data_pred
    else:
        filtered_data = data_pred[data_pred['boro_nm'].str.contains(user_input)]
        return filtered_data
        print(user_input)

In [23]:
user_input = input("Enter boron name: ")
print(user_input)
m = filter_data(data_pred, user_input)
m

Enter boron name: QUEENS
QUEENS


,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
2,40.596465,-73.781955,QUEENS,2021.0,4.0,3.0,22.0,50.0,MISDEMEANOR,HARRASSMENT 2
9,40.714382,-73.776656,QUEENS,2021.0,6.0,3.0,12.0,20.0,MISDEMEANOR,HARRASSMENT 2
11,40.758099,-73.828809,QUEENS,2021.0,1.0,3.0,8.0,0.0,MISDEMEANOR,PETIT LARCENY
15,40.589920,-73.788691,QUEENS,2021.0,3.0,2.0,17.0,25.0,MISDEMEANOR,HARRASSMENT 2
26,40.690403,-73.820433,QUEENS,2021.0,11.0,1.0,7.0,50.0,MISDEMEANOR,HARRASSMENT 2
...,...,...,...,...,...,...,...,...,...,...
89880,40.745253,-73.870063,QUEENS,2018.0,8.0,6.0,0.0,1.0,FELONY,SEX CRIMES
89882,40.741871,-73.917066,QUEENS,2021.0,8.0,3.0,8.0,0.0,MISDEMEANOR,PETIT LARCENY
89889,40.751180,-73.915196,QUEENS,2021.0,5.0,0.0,21.0,30.0,MISDEMEANOR,HARRASSMENT 2
89891,40.694024,-73.851202,QUEENS,2021.0,11.0,0.0,18.0,0.0,MISDEMEANOR,PETIT LARCENY


In [24]:
def transposeList(l1):
    return list(map(list, zip(*l1)))

In [25]:
testLat = []
testLong = []
testBoro = []
# testBoro = {}
testData = []
for key in dict_boro_nm:
    temp = filter_data(data_pred, key)
    testLat = temp['latitude'].unique()
    testLong = temp['longitude'].unique()
    testBoro = [key] * len(testLat)
#     testBoro[key] = {1: testLat,  \
#                      2: testLong}
#     temp = np.array([testLat, testLong, testBoro])
    testData += transposeList([testLat, testLong, testBoro])

In [26]:
testData

[[40.84583860700008, -73.93446226999998, 'OTHERS'],
 [40.73389675800007, -74.005395837, 'OTHERS'],
 [40.88202340700008, -73.86620093899995, 'OTHERS'],
 [40.63785585600005, -73.91843818899997, 'OTHERS'],
 [40.74344586700005, -74.00169758699997, 'OTHERS'],
 [40.57758741600002, -73.97633607099993, 'OTHERS'],
 [40.84457729200005, -73.91530776399996, 'OTHERS'],
 [40.67035648500007, -73.89742181299994, 'OTHERS'],
 [40.81269913700004, -73.92161973599997, 'OTHERS'],
 [40.795581553000034, -73.93229781499997, 'OTHERS'],
 [40.77504198300005, -73.98206801399994, 'OTHERS'],
 [40.75461661500003, -73.90883496299993, 'OTHERS'],
 [40.64997733000007, -73.94756329099994, 'OTHERS'],
 [40.77021126900007, -73.98224260599994, 'OTHERS'],
 [40.841448077000045, -73.90445130899997, 'OTHERS'],
 [40.81427840300007, -73.91534270799998, 'OTHERS'],
 [40.62141559900005, -74.08007850499997, 'OTHERS'],
 [40.73669094600008, -73.99523109199998, 'OTHERS'],
 [40.80148755200002, -73.96036389799998, 'OTHERS'],
 [40.8551305660

In [27]:
len_testData = len(testData)

In [28]:
import random
from datetime import datetime

random.shuffle(testData)
testData

[[40.70672930300003, -73.74990442999997, 'QUEENS'],
 [40.66188362600008, -73.73817081199996, 'QUEENS'],
 [40.78694845900003, -73.97861644799997, 'MANHATTAN'],
 [40.70807749100004, -73.78259233799997, 'QUEENS'],
 [40.868182313000034, -73.84667212299998, 'BRONX'],
 [40.661303780000026, -73.97548527299993, 'BROOKLYN'],
 [40.60431285400006, -73.95551117699993, 'BROOKLYN'],
 [40.586936736000034, -73.89230561199997, 'QUEENS'],
 [40.63664123700005, -73.99402762099999, 'BROOKLYN'],
 [40.72198571600006, -73.98039030299998, 'MANHATTAN'],
 [40.64003031600004, -73.90828888299995, 'BROOKLYN'],
 [40.668986493000034, -73.94501418999994, 'BROOKLYN'],
 [40.67526902700007, -73.91953154799995, 'BROOKLYN'],
 [40.86012856000008, -73.901210613, 'BRONX'],
 [40.859877246000046, -73.89631972299998, 'BRONX'],
 [40.84074193000004, -73.92165536399993, 'BRONX'],
 [40.60849903700006, -74.00112871299996, 'BROOKLYN'],
 [40.67712320100002, -73.97076314599997, 'BROOKLYN'],
 [40.87168982500003, -73.88414147999998, 'BRON

In [29]:
testTime = pd.DataFrame({'timestamp': pd.date_range(start="2022-03-01", end="2022-08-01", periods=len_testData)})
testTime['timestamp'] = testTime['timestamp'].astype(str)
testTime[['cmplnt_fr_dt','cmplnt_fr_tm']] = testTime['timestamp'].str.split(" ", 1, expand=True)
del testTime['timestamp']
testTime

,cmplnt_fr_dt,cmplnt_fr_tm
0,2022-03-01,00:00:00.000000000
1,2022-03-01,00:06:03.184790373
2,2022-03-01,00:12:06.369580746
3,2022-03-01,00:18:09.554371119
4,2022-03-01,00:24:12.739161492
...,...,...
36394,2022-07-31,23:35:47.260838508
36395,2022-07-31,23:41:50.445628880
36396,2022-07-31,23:47:53.630419254
36397,2022-07-31,23:53:56.815209626


In [30]:
testLocs = pd.DataFrame(testData, columns=['latitude', 'longitude', 'boro_nm'])
testLocs

,latitude,longitude,boro_nm
0,40.706729,-73.749904,QUEENS
1,40.661884,-73.738171,QUEENS
2,40.786948,-73.978616,MANHATTAN
3,40.708077,-73.782592,QUEENS
4,40.868182,-73.846672,BRONX
...,...,...,...
36394,40.649954,-73.905392,BROOKLYN
36395,40.674834,-73.898576,BROOKLYN
36396,40.876154,-73.877328,BRONX
36397,40.593866,-73.970857,BROOKLYN


In [31]:
testDataset = testTime.merge(testLocs, left_index=True, right_index=True)

In [32]:
testDataset

,cmplnt_fr_dt,cmplnt_fr_tm,latitude,longitude,boro_nm
0,2022-03-01,00:00:00.000000000,40.706729,-73.749904,QUEENS
1,2022-03-01,00:06:03.184790373,40.661884,-73.738171,QUEENS
2,2022-03-01,00:12:06.369580746,40.786948,-73.978616,MANHATTAN
3,2022-03-01,00:18:09.554371119,40.708077,-73.782592,QUEENS
4,2022-03-01,00:24:12.739161492,40.868182,-73.846672,BRONX
...,...,...,...,...,...
36394,2022-07-31,23:35:47.260838508,40.649954,-73.905392,BROOKLYN
36395,2022-07-31,23:41:50.445628880,40.674834,-73.898576,BROOKLYN
36396,2022-07-31,23:47:53.630419254,40.876154,-73.877328,BRONX
36397,2022-07-31,23:53:56.815209626,40.593866,-73.970857,BROOKLYN


In [34]:
testDataset.to_csv('testDataset.csv')

In [35]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,min_samples_split=100)
rfc.fit(x_train, y_train)

RandomForestClassifier(min_samples_split=100)

In [36]:
rfc_pred = rfc.predict(x_test)
print(classification_report(y_test[:,0],rfc_pred[:,0]))

              precision    recall  f1-score   support

           0       0.51      0.12      0.20     29890
           1       0.51      0.94      0.66     45116
           2       0.60      0.00      0.00     14894

    accuracy                           0.51     89900
   macro avg       0.54      0.35      0.29     89900
weighted avg       0.53      0.51      0.40     89900



In [37]:
print(classification_report(y_test[:,1],rfc_pred[:,1]))

              precision    recall  f1-score   support

           0       0.39      0.02      0.04       840
           1       0.30      0.67      0.41     17485
           2       0.26      0.10      0.14      9018
           3       0.18      0.15      0.17      9581
           4       0.32      0.13      0.19      8157
           5       0.00      0.00      0.00      3442
           6       0.00      0.00      0.00      1686
           7       0.21      0.49      0.30     14792
           8       0.41      0.12      0.19      1322
           9       0.00      0.00      0.00       290
          10       0.23      0.00      0.00      3046
          11       0.87      0.90      0.89        92
          12       0.27      0.02      0.03       510
          13       0.31      0.01      0.02       900
          14       0.26      0.02      0.04      2596
          15       0.00      0.00      0.00       256
          16       0.24      0.01      0.02      4502
          17       0.00    

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [38]:
# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(x_train, y_train)

DecisionTreeClassifier()

In [39]:
dtree_pred = dtree.predict(x_test)
dtree_pred

array([[ 0, 14],
       [ 0,  8],
       [ 1,  1],
       ...,
       [ 0,  4],
       [ 0, 16],
       [ 0, 19]], dtype=int64)

In [40]:
print(classification_report(y_test[:,0],dtree_pred[:,0]))

              precision    recall  f1-score   support

           0       0.39      0.39      0.39     29890
           1       0.55      0.54      0.54     45116
           2       0.23      0.24      0.24     14894

    accuracy                           0.44     89900
   macro avg       0.39      0.39      0.39     89900
weighted avg       0.44      0.44      0.44     89900



In [41]:
print(classification_report(y_test[:,1],dtree_pred[:,1]))

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

              precision    recall  f1-score   support

           0       0.08      0.07      0.07       840
           1       0.40      0.38      0.39     17485
           2       0.19      0.18      0.19      9018
           3       0.20      0.21      0.21      9581
           4       0.17      0.16      0.17      8157
           5       0.07      0.07      0.07      3442
           6       0.05      0.05      0.05      1686
           7       0.23      0.24      0.24     14792
           8       0.44      0.47      0.45      1322
           9       0.10      0.11      0.10       290
          10       0.08      0.07      0.07      3046
          11       0.85      0.82      0.83        92
          12       0.09      0.09      0.09       510
          13       0.09      0.08      0.09       900
          14       0.08      0.08      0.08      2596
          15       0.04      0.04      0.04       256
          16       0.14      0.15      0.15      4502
          17       0.00    

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
